In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%autosave 0
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

import sys
sys.path.insert(0,'..')

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')


Autosave disabled


In [2]:
# LOAD DATA
#{seqname:[shapearray, sequence, structure]}

import rna_io
data = rna_io.get_all_data('../data/RNA16.react', '../data/RNA16.dbn')
data.pop("R009")
data.pop("23sRNA")
'''
data.pop("ADDRSW")
data.pop("GLYCFN")
data.pop("RNAsep")
data.pop("ZHCV")
data.pop("adenin")
'''
print "done"


data for 'tRNA-asp' is corrupted, ignoring...
done


In [3]:
# TRAIN ON N-1, predict on rest
import simushape_representatives as sr
import rna_tools
import collections
import matplotlib.pyplot as plt
from scipy.stats import pearsonr as pear
import sukosd
import rna_io
import numpy as np


data = rna_io.get_all_data('../data/RNA16.react', '../data/RNA16.dbn') # for Sukosd

#predictions = list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.rnashapes, cutoff=0.0000000001))


def run_all_analysis(cross_predictions_list,fig_title, Sukosd=None):
    
    data.pop("R009",None) # we use this with the same trainig set to call run_all_analysis(something, 'Realstructure-Shaker-Z-sametrain')
    data.pop("23sRNA",None) # we use this with the same trainig set to call run_all_analysis(something, 'Realstructure-Shaker-Z-sametrain')
    pmfereallist = []                                                                                                                                                      
    pmfesimulist = []
    psimureallist = []
    keys= data.keys()
    print "keys : ", keys
    
    # FOR ALL seqyences
    for i, key in enumerate(keys):
        # GET ACCessibility for sequence
        mfelist = rna_tools.soheilas_plfold(data[key][1])# Probability of each nucleotide in sequence (data[key][1]) 
        # GET accessibility with ... sequence and real SHAPE data
        realShapelist = rna_tools.soheilas_plfold(data[key][1], data[key][0]) 
        #GET ACCESSIBILITY sequcence and simulated shape data
        if Sukosd == None:
            simuShapelist = rna_tools.soheilas_plfold(data[key][1], cross_predictions_list[i]) #name of reactvity file 
            
        # compute pearson correlation between probabilities computed between MFE, simu, realshape
        else: simuShapelist = rna_tools.soheilas_plfold(data[key][1], cross_predictions_list[key])
        pmfereal = pear(mfelist,realShapelist)[0]
        pmfesimu = pear(mfelist,simuShapelist)[0]
        psimureal = pear(simuShapelist,realShapelist)[0]
        #list of all MFE (simu or real) of all sequences
        pmfereallist.append(pmfereal)
        pmfesimulist.append(pmfesimu)
        psimureallist.append(psimureal)

       
    print 
    print ('*******  ', fig_title, '******* ')
    print "Results: ENS&realShape ENS&simu simu&realShape"
    print

    #print correlations with RNA names
    for i, key in enumerate(keys):
        print "%s\t\t\t%0.2f\t%0.2f\t%0.2f" % (key, pmfereallist[i], pmfesimulist[i], psimureallist[i])
    
    
     #*******************************+
    corrData = [pmfereallist, pmfesimulist, psimureallist]
    
    fig = plt.figure(figsize=(8,6))

    plt.boxplot(corrData, 0, 'rs', 1)
    plt.xticks([y+1 for y in range(len(corrData))], ['ENS&RealShape', 'ENS&SimuShape', 'SimuShape&RealShape'])
    plt.xlabel(fig_title)
    plt.ylim([0, 1])
    #*************************************1111111111111111
    
    ax = fig.add_subplot(1, 1, 1)

    # Major ticks every 20, minor ticks every 5
    major_ticks = np.arange(0, 1, 0.05)
    minor_ticks = np.arange(0, 1, 0.05)

    #ax.set_xticks(major_ticks)
    #ax.set_xticks(minor_ticks, minor=True)
    ax.set_yticks(major_ticks)
    ax.set_yticks(minor_ticks, minor=True)

    # And a corresponding grid
    ax.grid(which='both')

    # Or if you want different settings for the grids:
    ax.grid(which='minor', alpha=0.2)
    ax.grid(which='major', alpha=0.2)
    
    plt.boxplot(corrData, 0, 'rs', 1)
    plt.scatter([1,2,3], [np.mean(x) for x in corrData] )
    plt.xticks([y+1 for y in range(len(corrData))], ['ENS&RealShape', 'ENS&SimuShape', 'SimuShape&RealShape'])
    plt.xlabel(fig_title)
    plt.ylim([0, 1])
    
    
    
    #plt.grid(True, axis='y', alpha=0.05)
        
    #plt.grid(True, linestyle='-', which='major', color='lightgrey',alpha=0.5)
    #plt.grid(True, axis='y')
    #*****************************************11111111111111111
    
    t = plt.title('Correlations between 2 of 3 ENS, SimuShape, RealShape')
    fig.savefig(fig_title + '-Correlations.png', bbox_inches='tight')
    plt.show()
    
     #*******************************+
        
        
    #4*4*******************************
    
    #for i, key in enumerate(keys):
     #   print "cross_predictions_list[i]", cross_predictions_list[i]
            
    j=0
    k=0
    fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(20, 30))
    for i, key in enumerate(keys):
        if i==0: k=0
        elif (i!=0)&(i%3==0):
            j=j+1
            k=0
        else: k=k+1
        print j, k, 'j, k'           
        mydata = [x for x in data[key][0] if x != None]
        if Sukosd !=None:
            ddata=[mydata, cross_predictions_list[key]]
        else: ddata=[mydata, cross_predictions_list[i]]
        axes[j, k].boxplot(ddata, labels=['RealShape', 'SimuShape'])
        axes[j, k].set_title('Comparison of SHAPE computed by (' + fig_title + ') for ' + key + 'length of' + str(len(cross_predictions_list[i]))) #Comparison of reactivities of RealShape and SimuShape (computed by ' + fig_title + ') for ' + key
          
    plt.show()
    fig.savefig(fig_title + '-Reactivities.png', bbox_inches='tight')
    
    
     #4*4*******************************
   
'''
    #print "\t\t%0.2f\t%0.2f\t%0.2f" % (np.mean(np.array(pmfereallist)), np.mean(np.array(pmfesimulist)), np.mean(np.array(psimureallist)))
    for i, key in enumerate(keys):
        mydata = [x for x in data[key][0] if x != None]
        ddata=[mydata, cross_predictions_list[i]]
        mpl_fig = plt.figure(figsize=(8,6))
        plt.boxplot(ddata, 0, 'rs', 1)
        plt.xticks([y+1 for y in range(len(ddata))], ['RealShape', 'SimuShape'])
        plt.xlabel(key)
        t = plt.title('Comparison of reactivities of RealShape and SimuShape (computed by ' + fig_title + ')')
        plt.show()
   ''' 

    
#predictionsSoheilaRNAstructure = list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.soheilas_workaround, cutoff=0.0000000001))

# predictionsSoheila = list(sr.crosspredict(data, keys, seq_to_db_function= lambda seq:[rna_tools.fold(seq)], cutoff=0.0000000001))
#predictionsSoheila = list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.rnasubopt, cutoff=0.0000000001))
#print "predictionsSoheila  ", predictionsSoheila
#probabilitySoheila = list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.soheilas_plfold, cutoff=0.0000000001))


run_all_analysis(list(sr.crosspredict(data, data.keys, seq_to_db_function= lambda seq:[rna_tools.fold(seq)])), 'RNAfold')


#run_all_analysis(list(sr.crosspredictInter(data, keys, seq_to_db_function= lambda seq:[rna_tools.fold(seq)])), 'RNAfold')#intermediate



# need function sequence -> real structure
#  {seqname:[react, sequence, dotbracketstring]}

trick= { sequence : [stru] for react, sequence, stru in data.values() }
get_real_struct = lambda seq : trick[seq] 
#run_all_analysis(list(sr.crosspredict(data, keys, seq_to_db_function = get_real_struct)), 'Realstructure-Shaker-Z')
something = {a:b for a,b in sr.modelpredict(data, keys)}

#run_all_analysis(something, 'Realstructure-Shaker-Z-sametrain', 1) #change the model for training



'''
#Sukosd with RNAfold
SukosdShapeList = []
for i, key in enumerate(keys):
    structure = rna_tools.fold(data[key][1])
    #print "structure : ", structure
    SukosdShape = sukosd.sukosd(structure)
    #print SukosdShape
    SukosdShapeList.append(SukosdShape)
print SukosdShapeList
run_all_analysis(SukosdShapeList, 'RNAfold') 
'''

'''
#Sukosd with real strutcure
SukosdShapeList = {}
for i, key in enumerate(keys):
    structure = data[key][2]
    #print "structure : ", structure
    SukosdShape = sukosd.sukosd(structure)
    #print SukosdShape
    SukosdShapeList[key]= SukosdShape
print SukosdShapeList

run_all_analysis(SukosdShapeList, 'RealStructure-Sukosd-Z') 
'''

'''
This is not logical.
averaged mean is necessary 
for i, key in enumerate(keys):
    structures = rna_tools.rnasubopt(data[key][1])
    #print "structures : ", structures
    for j, structure in enumerate(structures):
        print "i : ", i
        print "j : ", j
        print "structure : ", structure
        SukosdShape = sukosd.sukosd(structure)
        print SukosdShape
run_all_analysis(SukosdShape, 'RNAsubopt') 
'''




data for 'tRNA-asp' is corrupted, ignoring...


NameError: name 'keys' is not defined

In [ ]:
print trick

In [ ]:

run_all_analysis(list(sr.crosspredict(data, keys, seq_to_db_function=rna_tools.rnasubopt)), 'RNAsubopt')



In [ ]:
asdkja
asdkkjsa
asda


In [ ]:
import rna_tools as rt

# EVALUATE 
for key,prediction in zip(keys,predictionsSoheila):
    proba = rt.probability(data[key][2],data[key][1])
    annproba = rt.probability(data[key][2],data[key][1], prediction)
    realReactProb = rt.probability(data[key][2],data[key][1],data[key][0])
    print "%s\t%0.9f\t%0.9f\t%0.9f\t%d\t%d\t%d" % (key, proba, annproba,  realReactProb, proba < annproba or realReactProb < annproba, data[key][2] == rt.fold(data[key][1]), realReactProb < annproba)
    

In [ ]:
# todo: subopt, regressor